# Setup Sims

We instantiate 300 networks, run each 20 times, yielding an original batch of 6,000 simulations
that will be re-analyzed and re-run throughout the paper (see Methods, simulations).


In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.dpi'] = 300.

In [ ]:
from tctx import simsetup
from tctx.analysis import simbatch as sb
from tctx import postpro

In [ ]:
iwm_range = (50, 110)
iws_range = (0, 120)

# Original batch

In [ ]:
%%time

input_batch_path = simsetup.setup_original_batch(
    'nets_2021.12.01',
    'original',
    net_count=300,
    sim_count_per_net=20,
    iwm_range=iwm_range,
    iws_range=iws_range,
)

In [ ]:
input_batch = sb.SimBatch.load(input_batch_path)

input_batch.describe_reg()

In [ ]:
!ls -lha ../../../data/interim/nets_*
!ls -lha ../../../data/interim/batch_*

## Post-process

In [ ]:
# Look up the results of executing seqexp.py
results_batch_path = sb.abs_path('seqexp/2021.12.01.00.00.00.000000/sweeps.h5')
!ls -lh $results_batch_path

batch_results = sb.SimBatch.load(results_batch_path).sel(status='success')
simsetup.plot_input_param_sampling(batch_results, iwm_range, iws_range)
batch_results.describe_reg()

In [ ]:
working_filename = sb.abs_path('batch_2021.12.01_processed/registry.h5')
seqs_folder = working_filename.parent / 'seqs'

simsetup.save_batch(batch_results, working_filename)

Register the results of the simulations and extract basic stats like firing rates

In [ ]:
postpro.register_raw(working_filename)

postpro.extract_simstats(working_filename)

Extract $\Delta$ FR for all cells, tag followers, and extract sequence stats

In [ ]:
postpro.extract_followers(working_filename, seqs_folder)

In [ ]:
postpro.extract_extents(working_filename)

In [ ]:
postpro.extract_order_entropy(working_filename, seqs_folder / 'order_entropy.h5')

Store follower cells and follower-to-follower connections separately for quicker access

In [ ]:
postpro.extract_all_foll_cells(working_filename, seqs_folder / 'foll_cells.h5')

postpro.extract_net_foll_conns(working_filename, seqs_folder / 'net_foll_conns.h5')

postpro.extract_str_foll_conns(working_filename, seqs_folder / 'str_foll_conns.h5', mask_col=None)

Extract activation matrices of followers and cluster using k-modes for Fig. 4

In [ ]:
postpro.extract_foll_amats(working_filename, seqs_folder / 'amat.h5')

In [ ]:
postpro.extract_kmodes_labels(working_filename, seqs_folder / f'kmode_labels.h5')

In [ ]:
sb.SimBatch.load(working_filename).describe_reg()

# Spike transfer extraction

In [ ]:
working_filename = sb.abs_path('batch_2021.12.01_processed/registry.h5')

batch_results = sb.SimBatch.load(working_filename)

## Randomly tag simulations for extraction

In [ ]:
motif_sim_count = 900
simsetup.randomly_tag(batch_results, motif_sim_count, 'uniform_sampling_sub')

simsetup.plot_input_param_sampling(
    batch_results,
    iwm_range,
    iws_range,
    highlight='uniform_sampling_sub',
)

In [ ]:
simsetup.save_batch(batch_results, working_filename)

## Post-process

In [ ]:
postpro.extract_all_jumps(
    working_filename,
    seqs_folder / 'jumps.h5',
    mask_col='uniform_sampling_sub',
)

In [ ]:
postpro.extract_motifs(
    working_filename,
    seqs_folder / 'motifs.h5',
    mask_col='uniform_sampling_sub',
)